In [593]:
import pandas as pd
import numpy as np
import copy

In [594]:
# Read Excel file into a DataFrame.
df = pd.read_excel('data.xlsx')

# Remove column.
df = df.drop(columns=['Unnamed: 0'])

# Change column name.
df = df.rename(columns={'Unnamed: 19': 'Reference'})

# Create column of reference with A and I.
df_ref_column = df['Reference']

# Store the DataFrame reference of A's and I's in a int.
total_A = df["Reference"].value_counts()["A"]
total_I = df["Reference"].value_counts()["I"]

# Define operative DataFrame:
# with the columns from X11 to X92, without Reference column.
df_op = df.loc[:, "X11": "X92"]

# Count the number of operatives columns (between X11 and X92).
nbr_operatives_columns = len(df_op.columns)

df.head()

,X11,X12,X21,X22,X31,X32,X41,X42,X51,X52,X61,X62,X71,X72,X81,X82,X91,X92,Reference
0,1,0,0,1,1,0,0,1,0,1,1,0,0,1,1,0,1,0,A
1,0,1,0,1,1,0,0,1,0,1,1,0,0,1,1,0,1,0,A
2,1,0,1,0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,I
3,1,0,0,1,0,1,1,0,1,0,0,1,1,0,0,1,0,1,I
4,0,1,0,1,1,0,0,1,0,1,1,0,0,1,0,1,1,0,A


In [595]:
# Create firstborns (chromosomes).

# Set the seed for reproducibility
#np.random.seed(42)

# Generate 6 random chromosomes.
chromosome1 = -1 + 2 * np.random.random(nbr_operatives_columns)
chromosome2 = -1 + 2 * np.random.random(nbr_operatives_columns)
chromosome3 = -1 + 2 * np.random.random(nbr_operatives_columns)
chromosome4 = -1 + 2 * np.random.random(nbr_operatives_columns)
chromosome5 = -1 + 2 * np.random.random(nbr_operatives_columns)
chromosome6 = -1 + 2 * np.random.random(nbr_operatives_columns)

# Create re-usable DataFrame with the current lineage
df_current_lineage = pd.DataFrame({'chromo_1':chromosome1,
								   'chromo_2':chromosome2,
								   'chromo_3':chromosome3,
								   'chromo_4':chromosome4,
								   'chromo_5':chromosome5,
								   'chromo_6':chromosome6})

df_current_lineage.head()

,chromo_1,chromo_2,chromo_3,chromo_4,chromo_5,chromo_6
0,-0.857573,0.197166,0.098373,-0.243679,0.982817,0.974240
1,-0.227862,0.208457,0.175671,-0.004324,0.809338,0.360816
2,0.452330,-0.430578,0.206853,-0.425894,0.230064,0.434864
3,-0.621942,-0.078066,0.341231,-0.578851,0.940666,-0.504851
4,-0.984366,0.071584,0.940552,0.273660,0.762486,-0.281109


In [596]:
# Function to aply each chromosome for each line of the DataFrame
# and return the result (line * chromosome).sum().
def chromo_action(row, chromosome, trash):
	res_mult = row * chromosome
	res_sum = res_mult.sum()
	return (res_sum)

# Iterate df_current_lineage in database.
i = 0
trash = None
df_chromo_result = pd.DataFrame()
max_i = len(df_current_lineage.columns)
while i < max_i:
	current_chromosome = (df_current_lineage.iloc[:, i]).values
	chromo_result = df_op.apply(func=chromo_action, axis=1, args=(current_chromosome, trash))
	df_chromo_result[f'Chromo {i+1} result'] = chromo_result
	i = i + 1

# Create chromo reference: a DataFrame that contain
# 'A' if number > 0 and 'I' if number < 0.
df_chromo_result = df_chromo_result.applymap(lambda x: 'A' if x > 0 else 'I')

# Calculate how many A's and how many I's were correct.
def chromo_count_matches(column):
	relative_A = ((column == 'A') & (df_ref_column == 'A')).sum()
	relative_I = ((column == 'I') & (df_ref_column == 'I')).sum()
	return pd.Series({'Right A': relative_A, 'Right I': relative_I})

df_chromo_result = df_chromo_result.apply(chromo_count_matches)

# Calculate the fitness for all the six chromosome
def fitness_calculation(column):
	relative_numerator = np.prod(column)
	fitness = relative_numerator / (total_A * total_I)
	return fitness

fitness_values = df_chromo_result.apply(fitness_calculation)
df_chromo_result.loc['Fitness'] = fitness_values

# Removing obsolete A and I rows
i_remove = ['Right A', 'Right I']
df_chromo_result = df_chromo_result.drop(i_remove)

df_chromo_result

,Chromo 1 result,Chromo 2 result,Chromo 3 result,Chromo 4 result,Chromo 5 result,Chromo 6 result
Fitness,0.0,0.092105,0.0,0.0,0.578947,0.0


## Raffle

In [597]:
# Get two raffle points to choose two chromosomes.
# The choosen chromosomes are going to be crossed.

# Create array with fitness values.
fitness_array = (df_chromo_result.iloc[0, :]).values

# Sum the array.
fitness_array_sum = fitness_array.sum()

# Create new array with cumulative sum.
norm_fit_0 = round((fitness_array[0]/ fitness_array_sum) * 100)
norm_fit_1 = round((fitness_array[1]/ fitness_array_sum) * 100) + norm_fit_0
norm_fit_2 = round((fitness_array[2]/ fitness_array_sum) * 100) + norm_fit_1
norm_fit_3 = round((fitness_array[3]/ fitness_array_sum) * 100) + norm_fit_2
norm_fit_4 = round((fitness_array[4]/ fitness_array_sum) * 100) + norm_fit_3
norm_fit_5 = round((fitness_array[5]/ fitness_array_sum) * 100) + norm_fit_4
norm_fitness_array = np.array([norm_fit_0, norm_fit_1, norm_fit_2, norm_fit_3, norm_fit_4, norm_fit_5])

# Sum the new array (this value must be equal to one).
# This is not used, is just a check.
norm_fitness_array_sum = norm_fitness_array.sum()

# raffle function: it resturn a position given a raffle number.
def get_raffle_point(raffle):
	max_i = len(norm_fitness_array)
	i = 0
	while i < max_i:
		if 0 <= raffle <= norm_fitness_array[0]:
			return 0
		elif norm_fitness_array[i] < raffle <= norm_fitness_array[i + 1]:
			return i
		i = i + 1

# Get two raffle points given a random raffle.
raffle_1 = round(np.random.random() * 100)
raffle_point_1 = get_raffle_point(raffle_1)
raffle_2 = round(np.random.random() * 100)
raffle_point_2 = get_raffle_point(raffle_2)

# Selec (from the random above) a mother and a father chromosome.
father_chromosome = df_current_lineage.iloc[:, raffle_point_1]
mother_chromosome = df_current_lineage.iloc[:, raffle_point_2]

## Cross

In [598]:
father_chromosome.values

array([-0.85757263, -0.22786243,  0.4523301 , -0.62194174, -0.98436626,
       -0.89571131,  0.68288294, -0.81840889, -0.47625194, -0.80785409,
        0.6729939 , -0.23878568, -0.5133896 , -0.60217161,  0.50646545,
        0.0468578 ,  0.26217553, -0.43498805])

In [599]:
mother_chromosome.values

array([-0.24367859, -0.00432446, -0.4258944 , -0.57885096,  0.2736605 ,
        0.75752232, -0.84818245, -0.4358776 ,  0.43919363, -0.45474643,
        0.3395481 , -0.65486704, -0.7613765 , -0.24858085,  0.02821759,
        0.31927764, -0.79899993, -0.41816324])

In [600]:
# cross function: it cross a father and a mother and generates a new born that
# is part mother and part father. The crossing point is random.
def cross_father_mother(father, mother):
	cross_point = round(17 * np.random.random())
	paternal_sperm  = father[cross_point:]
	maternal_egg = mother[:cross_point]
	new_born = np.hstack((maternal_egg, paternal_sperm))
	return (new_born)

# Create 3 new borns with the same father and mother.
new_born_1 = cross_father_mother(father_chromosome.values, mother_chromosome.values)
new_born_2 = cross_father_mother(father_chromosome.values, mother_chromosome.values)
new_born_3 = cross_father_mother(father_chromosome.values, mother_chromosome.values)

print(new_born_1)
print()
print(new_born_2)
print()
print(new_born_3)

[-0.24367859 -0.00432446 -0.4258944  -0.57885096  0.2736605   0.75752232
  0.68288294 -0.81840889 -0.47625194 -0.80785409  0.6729939  -0.23878568
 -0.5133896  -0.60217161  0.50646545  0.0468578   0.26217553 -0.43498805]

[-0.24367859 -0.00432446 -0.4258944  -0.57885096  0.2736605  -0.89571131
  0.68288294 -0.81840889 -0.47625194 -0.80785409  0.6729939  -0.23878568
 -0.5133896  -0.60217161  0.50646545  0.0468578   0.26217553 -0.43498805]

[-0.24367859 -0.00432446 -0.4258944  -0.62194174 -0.98436626 -0.89571131
  0.68288294 -0.81840889 -0.47625194 -0.80785409  0.6729939  -0.23878568
 -0.5133896  -0.60217161  0.50646545  0.0468578   0.26217553 -0.43498805]


## Mutate

In [601]:
df_current_lineage

,chromo_1,chromo_2,chromo_3,chromo_4,chromo_5,chromo_6
0,-0.857573,0.197166,0.098373,-0.243679,0.982817,0.974240
1,-0.227862,0.208457,0.175671,-0.004324,0.809338,0.360816
2,0.452330,-0.430578,0.206853,-0.425894,0.230064,0.434864
3,-0.621942,-0.078066,0.341231,-0.578851,0.940666,-0.504851
4,-0.984366,0.071584,0.940552,0.273660,0.762486,-0.281109
5,-0.895711,0.927420,0.930720,0.757522,-0.508828,-0.641573
6,0.682883,0.917707,0.980837,-0.848182,-0.688153,0.176529
7,-0.818409,-0.882895,-0.074893,-0.435878,-0.468455,0.983379
8,-0.476252,-0.514413,0.497865,0.439194,-0.745001,-0.418458
9,-0.807854,-0.493368,0.774505,-0.454746,-0.513734,0.725470


In [602]:
# Get random value 0-5 to be the point of mutation.
def get_random_value(df_current_lineage):
	# Select a random row
	random_row = df_current_lineage.sample()
	# Select a random column
	random_column = np.random.choice(df_current_lineage.columns)
	# Get the value at the random row and random column
	random_value = random_row[random_column].values[0]
	return random_value

In [603]:
# Function to mutate a given new born.
def mutate_newborn(new_born, random_value):
	mutation_point = round(5 * np.random.random())
	mutated_new_born = copy.deepcopy(new_born)
	mutated_new_born[mutation_point] = random_value
	return mutated_new_born

In [604]:
# Mutating the three new borns
random_value = get_random_value(df_current_lineage)
mutated_new_born_1 = mutate_newborn(new_born_1, random_value)
random_value = get_random_value(df_current_lineage)
mutated_new_born_2 = mutate_newborn(new_born_2, random_value)
random_value = get_random_value(df_current_lineage)
mutated_new_born_3 = mutate_newborn(new_born_3, random_value)

print(mutated_new_born_1)
print(mutated_new_born_2)
print(mutated_new_born_3)

[-0.24367859  0.42517765 -0.4258944  -0.57885096  0.2736605   0.75752232
  0.68288294 -0.81840889 -0.47625194 -0.80785409  0.6729939  -0.23878568
 -0.5133896  -0.60217161  0.50646545  0.0468578   0.26217553 -0.43498805]
[-0.24367859 -0.00432446 -0.4258944  -0.57885096  0.2736605  -0.89571131
  0.68288294 -0.81840889 -0.47625194 -0.80785409  0.6729939  -0.23878568
 -0.5133896  -0.60217161  0.50646545  0.0468578   0.26217553 -0.43498805]
[-0.24367859 -0.00432446 -0.4258944  -0.62194174 -0.98436626  0.17567115
  0.68288294 -0.81840889 -0.47625194 -0.80785409  0.6729939  -0.23878568
 -0.5133896  -0.60217161  0.50646545  0.0468578   0.26217553 -0.43498805]
